In [5]:
import re

def parse_symbolic_hex(sym_hex_str: str):
    """
    Повертає:
    - total_expression: текстовий запис суми вигляду:
        X = <член1> + <член2> + ... + <членN>
    - constraints: список обмежень для змінних x_i (наприклад, "0 <= x1 < 16^2")

    sym_hex_str: рядок типу "0x12$$8e$$7$$" або "12$fe$"
    """
    # 1. Прибрати можливий префікс "0x"
    if sym_hex_str.lower().startswith("0x"):
        sym_hex_str = sym_hex_str[2:]

    # 2. Розбити на фрагменти:
    #    - "конкретні" шістнадцяткові (без $)
    #    - "символьні" (одне чи кілька поспіль $)
    # Для цього скористаємось регулярним виразом:
    #    [0-9a-fA-F]+  - послідовність звичайних шістнадцяткових цифр
    #    \$+          - послідовність символів $
    tokens = re.findall(r'[0-9a-fA-F]+|\$+', sym_hex_str)

    # 3. Порахуємо загальну кількість *ніблів* (4-бітових цифр),
    #    щоби знати степені 16, які множимо на кожний фрагмент.
    #    Для «конкретного» фрагмента довжиною L маємо L ніблів.
    #    Для «символьного» фрагмента довжиною m маємо теж m ніблів,
    #      але це будуть невідомі (x_i) у межах [0..16^m-1].
    total_nibbles = 0
    for t in tokens:
        if '$' in t:
            # m разів '$' -> m ніблів
            total_nibbles += len(t)
        else:
            # звичайний шістнадцятковий фрагмент -> довжина рядка в ніблах
            total_nibbles += len(t)

    # 4. Формуємо вираз X як суму. Для кожного токена зліва направо
    #    зменшуємо степінь 16 на кількість ніблів у цьому токені.
    expression_parts = []
    constraints = []
    var_index = 1  # Лічильник для x1, x2, x3, ...

    current_pos = 0  # рахуємо, скільки ніблів уже "пройдено" зліва направо

    for t in tokens:
        length = len(t)  # кількість ніблів (або символів $)

        # Позиція праворуч: це скільки ніблів *залишилось* після поточного
        # (current_pos визначає, скільки вже пройдено зліва).
        # Але для зручності можна обчислити степінь 16 "на льоту",
        # поступово рухаючись вправо.

        # Скільки ніблів іще залишиться *після* обробки цього токена?
        #     (total_nibbles - (current_pos + length))
        power = total_nibbles - (current_pos + length)

        if '$' in t:
            # Символьний фрагмент довжиною m = length
            var_name = f"x{var_index}"
            var_index += 1

            # Додаємо до виразу: var_name * (16^power)
            part_str = f"{var_name}*(16^{power})" if power > 0 else var_name
            expression_parts.append(part_str)

            # Формуємо обмеження 0 <= var_name < 16^m
            constraints.append(f"0 <= {var_name} < 16^{length}")

        else:
            # Конкретний шістнадцятковий фрагмент
            # Перетворимо його у форму 0x<t> для читабельності
            numeric_val_str = f"0x{t}"

            # Додаємо цей доданок з відповідною степінню 16
            if power > 0:
                part_str = f"{numeric_val_str}*(16^{power})"
            else:
                part_str = numeric_val_str

            expression_parts.append(part_str)

        current_pos += length

    # 5. Формуємо фінальний рядок для X
    total_expression = " + ".join(expression_parts)
    total_expression = "X = " + total_expression

    return total_expression, constraints


if __name__ == "__main__":
    # Приклад:
    sym_str = "0x12$$8e$$7$$"

    expr, constrs = parse_symbolic_hex(sym_str)

    print("Вираз:")
    print(expr)
    print("\nОбмеження для змінних:")
    for c in constrs:
        print(c)


Вираз:
X = 0x12*(16^9) + x1*(16^7) + 0x8e*(16^5) + x2*(16^3) + 0x7*(16^2) + x3

Обмеження для змінних:
0 <= x1 < 16^2
0 <= x2 < 16^2
0 <= x3 < 16^2
